# FERNANDO LEON FRANCO

In [37]:
import numpy as np
import polars as pl
from bs4 import BeautifulSoup
import os
from colorstreak import Logger
import re
from dataclasses import dataclass


In [38]:
def print_bar(i, cantidad_registros, contexto="PROGRESO"):
    porcentaje = (i + 1) / cantidad_registros * 100
    # Con emojis
    barra = int(50 * (i + 1) / cantidad_registros) * "🟩"
    espacio = int(50 - len(barra)) * "⬛️"

    print(f"\r{contexto}: |{barra}{espacio}| {porcentaje:6.2f}%", end="", flush=True)

# MEJORAR LA CARGA DESDE UN CSV

In [39]:
def generar_csv(path_lectura, path_guardado):
    # Obtener la ruta absoluta del directorio actual
    # Leerlo y convertirlo en un DataFrame .csv para manipularlo mejor
    with open(path_lectura, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        
    data = [line.strip().split(":::") for line in lines]


    df = pl.DataFrame(data, schema=["id", "genero", "pais"], orient="row")


    # Guardar el DataFrame como un archivo .csv
    df.write_csv(path_guardado)
    return True

# ==================== Configuración de rutas =====================


base_path = os.getcwd()

# ===================== Creación de CSV de datos de prueba =====================
ruta_lectura_prueba = os.path.join(base_path, "data/author_profiling/es_test/truth.txt")
ruta_guardado_prueba = os.path.join(base_path, "data/author_profiling/es_test/truth_test.csv")

creado = generar_csv(ruta_lectura_prueba, ruta_guardado_prueba)

if creado:
    Logger.info(f'Archivo CSV creado en: "{ruta_guardado_prueba}"')
# ==============================================================================


# ===================== Creación de CSV de datos de entrenamiento ==============
ruta_lectura_entrenamiento = os.path.join(base_path, "data/author_profiling/es_train/truth.txt")
ruta_guardado_entrenamiento = os.path.join(base_path, "data/author_profiling/es_train/truth_train.csv")

creado_entrenamiento = generar_csv(ruta_lectura_entrenamiento, ruta_guardado_entrenamiento)
if creado_entrenamiento:
    Logger.info(f'Archivo CSV creado en: "{ruta_guardado_entrenamiento}"')
# ==============================================================================



[INFO] Archivo CSV creado en: "/Users/ferleon/Github/semestre_v/procesamiento_lenguaje/data/author_profiling/es_test/truth_test.csv"
[INFO] Archivo CSV creado en: "/Users/ferleon/Github/semestre_v/procesamiento_lenguaje/data/author_profiling/es_train/truth_train.csv"


# CARGAR LOS INDICES DESDE TRUTH DESDE EL CSV

In [40]:
def cargar_xml(id_archivo, train=True) -> str:
    ruta_base = os.path.join(base_path, "data/author_profiling/es_test" if not train else "data/author_profiling/es_train")
    ruta_archivo = os.path.join(ruta_base, f"{id_archivo}.xml")
    # Logger.info(f"ruta archivo: {ruta_archivo}")
    
    with open(ruta_archivo, 'r', encoding='utf-8') as file:
        xml_text = file.read()

    return xml_text

# ===================== Cargar CSV de datos  ==================
 
df_indices = pl.read_csv(ruta_guardado_entrenamiento)
print(df_indices.head())

cantidad_registros = len(df_indices)

registros_crudos = [("id_user","xml_doc","pais","genero") for i in range(cantidad_registros)]


for i, reg in enumerate(registros_crudos):
    id_archivo = df_indices['id'][i]
    id_user = id_archivo
    pais = df_indices['pais'][i]
    genero = df_indices['genero'][i]

    xml_text: str = cargar_xml(id_archivo)
    registros_crudos[i] = (id_user, xml_text, pais, genero)

    print_bar(i, cantidad_registros, contexto="CARGA XML")

print()
Logger.info("Carga de archivos XML completada.\n")
id_user, xml_text, pais, genero = registros_crudos[0]


Logger.debug(f"Total de tweets crudos cargados: {len(registros_crudos)} mostrando 1 registro:\n id:{id_user}\n pais:{pais}\n genero:{genero}\n xml_text:\n{xml_text[:50]}...\n")

shape: (5, 3)
┌─────────────────────────────────┬────────┬──────────┐
│ id                              ┆ genero ┆ pais     │
│ ---                             ┆ ---    ┆ ---      │
│ str                             ┆ str    ┆ str      │
╞═════════════════════════════════╪════════╪══════════╡
│ 74bcc9b0882c8440716ff370494aea… ┆ female ┆ colombia │
│ 4639c055f34ca1f944d0137a5aeb79… ┆ female ┆ colombia │
│ 92ffa98bade702b86417b118e8aca3… ┆ female ┆ colombia │
│ 4560c6567afcccef265f048ed117d0… ┆ female ┆ colombia │
│ 393866dfaa80d414c9896cf8723932… ┆ female ┆ colombia │
└─────────────────────────────────┴────────┴──────────┘
CARGA XML: |🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩| 100.00%
[INFO] Carga de archivos XML completada.

[DEBUG] Total de tweets crudos cargados: 4200 mostrando 1 registro:
 id:74bcc9b0882c8440716ff370494aea09
 pais:colombia
 genero:female
 xml_text:
<author lang="es">
	<documents>
		<document><![CDA...



# PROCESAMOS LOS TEXTOS XML Y LOS DEJAMOS EN UN DF

In [41]:
diccionario_idioma = {
    'es': 'Español', 
    'en': 'Inglés', 
    'fr': 'Francés', 
    'de': 'Alemán', 
    'it': 'Italiano', 
    'nl': 'Neerlandés'
}

def limpiar_xml(texto_xml:str):
    soup = BeautifulSoup(texto_xml, 'lxml-xml')   
    lang = str(soup.author.get('lang'))
    idioma = diccionario_idioma[lang] if lang in diccionario_idioma else lang
    
    documentos = soup.find_all('document') # Obtenemos todsa las etiquetas <document>
    tweets = [doc.get_text(separator=" ", strip=True) for doc in documentos] # Extraemos el texto de cada documento
    
    return tweets, idioma



# ===================== Procesamiento de limpieza del XML =====================
cantidad_registros = len(registros_crudos)

registros_procesados = []

for i, (id_user, doc_crudo, pais, genero) in enumerate(registros_crudos):
    lista_tweets_por_usuario, idioma = limpiar_xml(doc_crudo)
    
    for tweet in lista_tweets_por_usuario:
        registros_procesados.append({
            "id_user": id_user,
            "tweet_crudo": tweet,
            "pais": pais,
            "genero": genero,
            "idioma": idioma
        })
    print_bar(i, cantidad_registros, contexto="Progreso registros limpiados")

print()

# ===================== Creamos dataset persistente =====================

df_registros = pl.DataFrame(registros_procesados)
df_registros.write_parquet("data/author_profiling/registros_procesados.parquet")
Logger.info("Guardando archivo")

lf_registros = df_registros.lazy()

# ===================== Muestra =====================


muestra = lf_registros.limit(5).collect()
Logger.debug(f"Total de tweets procesados: {len(registros_procesados)} mostrando 5 primeros registros:\n {muestra}")


Progreso registros limpiados: |🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩| 100.00%
[INFO] Guardando archivo
[DEBUG] Total de tweets procesados: 419998 mostrando 5 primeros registros:
 shape: (5, 5)
┌─────────────────────────────────┬─────────────────────────────────┬──────────┬────────┬─────────┐
│ id_user                         ┆ tweet_crudo                     ┆ pais     ┆ genero ┆ idioma  │
│ ---                             ┆ ---                             ┆ ---      ┆ ---    ┆ ---     │
│ str                             ┆ str                             ┆ str      ┆ str    ┆ str     │
╞═════════════════════════════════╪═════════════════════════════════╪══════════╪════════╪═════════╡
│ 74bcc9b0882c8440716ff370494aea… ┆ Tiene que valer la pena que es… ┆ colombia ┆ female ┆ Español │
│ 74bcc9b0882c8440716ff370494aea… ┆ Tintas chinas, si ven ésto, es… ┆ colombia ┆ female ┆ Español │
│ 74bcc9b0882c8440716ff370494aea… ┆ "Maestro no le abrió!" -Ay qué… ┆ colombia ┆ female ┆ Espa


# Capa de dataclass de utilidd para normalización y limpieza de tweets

In [42]:
from nltk.tokenize import TweetTokenizer
import unicodedata
# ===================== Configuración de Limpieza de tweets =====================


@dataclass(frozen=True)
class ConfigLimpieza:
    normalizar_unicode: bool = True
    a_minusculas: bool = True
    quitar_urls: bool = True
    quitar_menciones: bool = True
    quitar_hashtags: bool = False 


    

@dataclass
class Tweet:
    URL = re.compile(r'https?://\S+', re.I)
    MENCION = re.compile(r'@\w+')
    HASHTAG = re.compile(r'#\w+')
    ESPACIO = re.compile(r'\s+')
    TT = TweetTokenizer()

    @staticmethod
    def limpiar(texto: str) -> str:
        cfg = ConfigLimpieza()
        if cfg.normalizar_unicode:
            texto = unicodedata.normalize("NFKC", texto)
        if cfg.a_minusculas:
            texto = texto.lower()
        if cfg.quitar_urls:
            texto = Tweet.URL.sub(" ", texto)
        if cfg.quitar_menciones:
            texto = Tweet.MENCION.sub(" ", texto)
        if cfg.quitar_hashtags:
            texto = Tweet.HASHTAG.sub(" ", texto)

        texto_limpio = Tweet.ESPACIO.sub(" ", texto).strip()
        return texto_limpio

    @staticmethod
    def tokenizar(texto: str) -> list[str]:
        return [t for t in Tweet.TT.tokenize(texto)]
    




# PIPELINE DE LIMPIEZA Y/O TOKENIZACIÓN DE TWEETS

In [43]:
# ===================== TRANSFORMACIONES EN PIPELINE =====================
"""
Recordemos que los pipelines de polars si es un lazyframe no se ejecutan hasta que se les pida con .collect() (Docoumentación)
"""


lf_registros_crudos = lf_registros


Logger.info("PIPELINE: LIMPIEZA/ TOKENIZACIÓN")
lf_limpio = (
    lf_registros_crudos.with_columns(
        pl.col("tweet_crudo")
          .map_elements(Tweet.limpiar)
          .alias("texto_limpio")
    )
)

Logger.debug(f"Mostrando registros del lazyframe limpio:\n {lf_limpio.limit(2).collect()}")

primer_tweet_limpio = (
    lf_limpio
    .select('texto_limpio')
    .limit(1)
    .collect()
    .to_series()
    .to_list()[0]
)

Logger.info(f"Primer tweet limpio: {primer_tweet_limpio}")
        
lf_tokens = (
    lf_limpio.with_columns(
        pl.col("texto_limpio")
          .map_elements(
              Tweet.tokenizar, 
              return_dtype=pl.List(pl.Utf8), 
              skip_nulls=True
            )
          .alias("tweet_tokenizado")
    )
)

Logger.debug(f"Mostrando registros del lazyframe tokenizado:\n {lf_tokens.limit(2).collect()}")


# ======================== Metodo para extraer el primer tweet tokenizado con polars =====================
primer_tweet_tokenizado = (
    lf_tokens
    .select("tweet_tokenizado") # Seleccionamos la columna que nos interesa
    .limit(1) 
    .collect()
    .item()  # Sirve para extraer el valor cuando el DataFrame es 1x1
)
# ========================================================================================================

Logger.info(f"Primer tweet tokenizado: {primer_tweet_tokenizado}")

for token in primer_tweet_tokenizado:
    Logger.info(f"Token: {token}")
    


[INFO] PIPELINE: LIMPIEZA/ TOKENIZACIÓN
[DEBUG] Mostrando registros del lazyframe limpio:
 shape: (2, 6)
┌──────────────────────┬──────────────────────┬──────────┬────────┬─────────┬──────────────────────┐
│ id_user              ┆ tweet_crudo          ┆ pais     ┆ genero ┆ idioma  ┆ texto_limpio         │
│ ---                  ┆ ---                  ┆ ---      ┆ ---    ┆ ---     ┆ ---                  │
│ str                  ┆ str                  ┆ str      ┆ str    ┆ str     ┆ str                  │
╞══════════════════════╪══════════════════════╪══════════╪════════╪═════════╪══════════════════════╡
│ 74bcc9b0882c8440716f ┆ Tiene que valer la   ┆ colombia ┆ female ┆ Español ┆ tiene que valer la   │
│ f370494aea…          ┆ pena que es…         ┆          ┆        ┆         ┆ pena que es…         │
│ 74bcc9b0882c8440716f ┆ Tintas chinas, si    ┆ colombia ┆ female ┆ Español ┆ tintas chinas, si    │
│ f370494aea…          ┆ ven ésto, es…        ┆          ┆        ┆         ┆ ven ésto,

# VAMOS A GENERAR UN VOCABULARIO GLOBAL

In [44]:
# ===================== VOCABULARIO GLOBAL =====================

Logger.info("CONSTRUCCIÓN DE VOCABULARIO GLOBAL")


lf_tokens_aplanados = (
    lf_tokens
    .select(
        'id_user', 
        pl.col('tweet_tokenizado')
    )
    .explode('tweet_tokenizado')
    .rename({'tweet_tokenizado':'tokens_del_tweet'})
)


lf_vocabulario = (
    lf_tokens_aplanados
    .group_by('tokens_del_tweet')
    .agg([
        pl.len().alias('frecuencia_global'),              # Conteo total de apariciones de cada token
        pl.n_unique('id_user').alias('documentos_unicos') # En cuántos documentos aparece ese token
    ])
)


Logger.debug(f"Mostrando registros del lazyframe tokenizado y aplanado:\n {lf_tokens_aplanados.collect()}")
Logger.debug(f"Mostrando vocabulario:\n {lf_vocabulario.collect()}")


[INFO] CONSTRUCCIÓN DE VOCABULARIO GLOBAL
[DEBUG] Mostrando registros del lazyframe tokenizado y aplanado:
 shape: (5_908_700, 2)
┌─────────────────────────────────┬──────────────────┐
│ id_user                         ┆ tokens_del_tweet │
│ ---                             ┆ ---              │
│ str                             ┆ str              │
╞═════════════════════════════════╪══════════════════╡
│ 74bcc9b0882c8440716ff370494aea… ┆ tiene            │
│ 74bcc9b0882c8440716ff370494aea… ┆ que              │
│ 74bcc9b0882c8440716ff370494aea… ┆ valer            │
│ 74bcc9b0882c8440716ff370494aea… ┆ la               │
│ 74bcc9b0882c8440716ff370494aea… ┆ pena             │
│ …                               ┆ …                │
│ 7ea28a182539c384d69cba3d10623f… ┆ recupere         │
│ 7ea28a182539c384d69cba3d10623f… ┆ ,                │
│ 7ea28a182539c384d69cba3d10623f… ┆ saludos          │
│ 7ea28a182539c384d69cba3d10623f… ┆ !                │
│ 7ea28a182539c384d69cba3d10623f… ┆ !        

In [45]:
# ===================== TR desde TU pipeline (con límites y CSR) =====================
# Requisitos previos en tu notebook:
# - lf_tokens_aplanados: ['id_user','tokens_del_tweet']
# - lf_vocabulario:      ['tokens_del_tweet','frecuencia_global','documentos_unicos']
# - lf_limpio:           para obtener el orden de documentos (id_user)
# - df_indices:          para etiquetas (pais)

import numpy as np
import polars as pl
from scipy.sparse import coo_matrix
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score

Logger.info("=== Ensamble mínimo: reusar vocabulario y tokens para TR (con límites y CSR) ===")

# -------------------- Parámetros de reducción de dimensión --------------------
MAX_TERMINOS = 50_000   # Top-K por frecuencia_global (ajústalo según tiempo/RAM)
MIN_DOCS     = 5        # Mínimo de documentos_unicos por token (descarta tokens ultra raros)

# 1) Orden estable de documentos (id_user) para alinear filas de TR e y_train
lf_docs = (
    lf_limpio
    .select(['id_user'])
    .unique(maintain_order=True)
    .sort('id_user')                    # si prefieres orden “natural”, puedes quitar este sort
    .with_row_count('indice_doc')       # índice de fila en TR
)
df_docs = lf_docs.collect()
orden_id_user = df_docs['id_user'].to_list()
num_docs = len(orden_id_user)
Logger.info(f"Total de documentos (usuarios): {num_docs:,}")

# 2) Etiquetas alineadas (y_train)
lf_etiquetas = (
    lf_docs.select(['id_user'])
    .join(
        df_indices.lazy().rename({'id':'id_user'}).select(['id_user','pais']),
        on='id_user',
        how='left'
    )
)
df_etiquetas = lf_etiquetas.collect()
y_train = df_etiquetas['pais'].to_list()
Logger.debug(f"Muestra de etiquetas (y): {y_train[:5]}")

# 3) Vocabulario reducido → diccionario de índices (token → columna)
Logger.info("Construyendo vocabulario reducido desde lf_vocabulario…")
lf_vocabulario_reducido = (
    lf_vocabulario
    .filter(pl.col('documentos_unicos') >= MIN_DOCS)      # filtra tokens raros
    .sort('frecuencia_global', descending=True)
    .limit(MAX_TERMINOS)                                   # tope duro Top-K
)

tokens_ordenados = (
    lf_vocabulario_reducido
    .select('tokens_del_tweet')
    .collect()['tokens_del_tweet']
    .to_list()
)
V = len(tokens_ordenados)
dicc_indices = {tok: j for j, tok in enumerate(tokens_ordenados)}
Logger.info(f"[DIM-RED] Vocabulario reducido: {V:,} términos (K={MAX_TERMINOS}, min_docs={MIN_DOCS})")

# 4) Conteos por (documento, token) y mapeo a índices de fila/columna (todo LazyFrame)
Logger.info("Contando frecuencias por (id_user, token) y mapeando a índices…")

# Tablas auxiliares como LazyFrame para joins LF↔LF
lf_tokens_permitidos = pl.DataFrame({'tokens_del_tweet': tokens_ordenados}).lazy()
lf_mapa_tokens = pl.DataFrame({
    'tokens_del_tweet': tokens_ordenados,
    'indice_token': list(range(V))
}).lazy()
lf_mapa_docs = lf_docs.select(['id_user','indice_doc'])

df_conteos = (
    lf_tokens_aplanados
    # filtra a tokens que están en el vocabulario reducido
    .join(lf_tokens_permitidos, on='tokens_del_tweet', how='inner')
    .group_by(['id_user','tokens_del_tweet'])
    .agg(pl.len().alias('frecuencia_en_documento'))
    # mapear id_user → indice_doc
    .join(lf_mapa_docs, on='id_user', how='inner')
    # mapear token → indice_token
    .join(lf_mapa_tokens, on='tokens_del_tweet', how='inner')
    .select(['indice_doc','indice_token','frecuencia_en_documento'])
    .collect()
)

Logger.info(f"Tripletas no-cero (indice_doc, indice_token, frecuencia): {df_conteos.height:,}")

# 5) Construcción de TR en formato disperso (CSR)
Logger.info("Construyendo TR en formato disperso (CSR)…")
filas = df_conteos['indice_doc'].to_numpy()
cols  = df_conteos['indice_token'].to_numpy()
vals  = df_conteos['frecuencia_en_documento'].to_numpy()

TR_frecuencia_csr = coo_matrix((vals, (filas, cols)), shape=(num_docs, V)).tocsr()
TR_binaria_csr    = TR_frecuencia_csr.copy()
TR_binaria_csr.data[:] = 1

Logger.info(f"[CSR] TR_frecuencia: shape={TR_frecuencia_csr.shape}, nnz={TR_frecuencia_csr.nnz:,}")
Logger.info(f"[CSR] TR_binaria:    shape={TR_binaria_csr.shape}, nnz={TR_binaria_csr.nnz:,}")

# 6) (Opcional) DOR del profe sobre TU TR reducida (si lo necesitas en el reporte)
def compute_dor_profe(TR_densa: np.ndarray) -> np.ndarray:
    DTR = np.zeros((TR_densa.shape[1], TR_densa.shape[0]), dtype=float)
    tam_v = TR_densa.shape[1]
    for i, doc in enumerate(TR_densa):
        pos_no_cero = np.nonzero(doc)[0]
        tam_vocab_doc = len(pos_no_cero)
        for termino in pos_no_cero:
            DTR[termino, i] = doc[termino] * np.log(tam_v / max(1, tam_vocab_doc))
    return DTR

# Nota: DOR requiere matriz densa. Si vas justo de tiempo/RAM, sáltalo o aplícalo sobre un subset.
# Ejemplo (solo si te alcanza memoria):
# TR_frecuencia_densa = TR_frecuencia_csr.toarray()
# TR_binaria_densa    = TR_binaria_csr.toarray()
# DTR_frecuencia = compute_dor_profe(TR_frecuencia_densa)
# DTR_binaria    = compute_dor_profe(TR_binaria_densa)
# Logger.info(f"DOR frecuencia: {DTR_frecuencia.shape} | DOR binaria: {DTR_binaria.shape}")

# 7) Normalización L2 directamente en CSR
Logger.info("Normalizando L2 (CSR)…")
TR_frecuencia_L2_csr = normalize(TR_frecuencia_csr, norm='l2', copy=True)
TR_binaria_L2_csr    = normalize(TR_binaria_csr,    norm='l2', copy=True)

# 8) Split 80/20 y entrenamiento con LinearSVC (rápido en alto-dim y disperso)
Logger.info("Split 80/20 estratificado (CSR)…")
Xf_tr, Xf_va, y_tr, y_va   = train_test_split(TR_frecuencia_csr,    y_train, test_size=0.2, stratify=y_train, random_state=42)
Xb_tr, Xb_va, _, _         = train_test_split(TR_binaria_csr,       y_train, test_size=0.2, stratify=y_train, random_state=42)
Xf_trL2, Xf_vaL2, _, _     = train_test_split(TR_frecuencia_L2_csr, y_train, test_size=0.2, stratify=y_train, random_state=42)
Xb_trL2, Xb_vaL2, _, _     = train_test_split(TR_binaria_L2_csr,    y_train, test_size=0.2, stratify=y_train, random_state=42)

Logger.info(f"Shapes → Frecuencia tr={Xf_tr.shape} va={Xf_va.shape} | Binaria tr={Xb_tr.shape} va={Xb_va.shape}")

def entrenar_y_evaluar(nombre_experimento, Xtr, Xva, ytr, yva):
    Logger.info(f"[LinearSVC] {nombre_experimento}")
    clf = LinearSVC(C=1.0)
    clf.fit(Xtr, ytr)
    pred = clf.predict(Xva)
    acc = accuracy_score(yva, pred)
    f1m = f1_score(yva, pred, average='macro')
    print(f"\n=== {nombre_experimento} ===")
    print(f"Accuracy (VA): {acc:.4f} | Macro-F1 (VA): {f1m:.4f}")
    return clf, acc, f1m

clf_bin,    acc_bin,    f1_bin    = entrenar_y_evaluar("Binario SIN L2",     Xb_tr,    Xb_va,    y_tr, y_va)
clf_fre,    acc_fre,    f1_fre    = entrenar_y_evaluar("Frecuencia SIN L2",  Xf_tr,    Xf_va,    y_tr, y_va)
clf_bin_l2, acc_bin_l2, f1_bin_l2 = entrenar_y_evaluar("Binario CON L2",     Xb_trL2,  Xb_vaL2,  y_tr, y_va)
clf_fre_l2, acc_fre_l2, f1_fre_l2 = entrenar_y_evaluar("Frecuencia CON L2",  Xf_trL2,  Xf_vaL2,  y_tr, y_va)

print("\n\n================= RESUMEN VALIDACIÓN (80/20) =================")
print(f"Binario SIN L2     -> Acc: {acc_bin:.4f} | Macro-F1: {f1_bin:.4f}")
print(f"Frecuencia SIN L2  -> Acc: {acc_fre:.4f} | Macro-F1: {f1_fre:.4f}")
print(f"Binario CON L2     -> Acc: {acc_bin_l2:.4f} | Macro-F1: {f1_bin_l2:.4f}")
print(f"Frecuencia CON L2  -> Acc: {acc_fre_l2:.4f} | Macro-F1: {f1_fre_l2:.4f}")

[INFO] === Ensamble mínimo: reusar vocabulario y tokens para TR (con límites y CSR) ===
[INFO] Total de documentos (usuarios): 4,200
[DEBUG] Muestra de etiquetas (y): ['spain', 'peru', 'venezuela', 'peru', 'spain']
[INFO] Construyendo vocabulario reducido desde lf_vocabulario…


/var/folders/j4/21ypxx3x0019wgj7rqnlm2340000gn/T/ipykernel_22901/3498009944.py:28: DeprecationWarning: `LazyFrame.with_row_count` is deprecated; use `LazyFrame.with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .with_row_count('indice_doc')       # índice de fila en TR


[INFO] [DIM-RED] Vocabulario reducido: 38,575 términos (K=50000, min_docs=5)
[INFO] Contando frecuencias por (id_user, token) y mapeando a índices…
[INFO] Tripletas no-cero (indice_doc, indice_token, frecuencia): 2,129,949
[INFO] Construyendo TR en formato disperso (CSR)…
[INFO] [CSR] TR_frecuencia: shape=(4200, 38575), nnz=2,129,949
[INFO] [CSR] TR_binaria:    shape=(4200, 38575), nnz=2,129,949
[INFO] Normalizando L2 (CSR)…
[INFO] Split 80/20 estratificado (CSR)…
[INFO] Shapes → Frecuencia tr=(3360, 38575) va=(840, 38575) | Binaria tr=(3360, 38575) va=(840, 38575)
[INFO] [LinearSVC] Binario SIN L2

=== Binario SIN L2 ===
Accuracy (VA): 0.9381 | Macro-F1 (VA): 0.9381
[INFO] [LinearSVC] Frecuencia SIN L2


/Users/ferleon/Github/semestre_v/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



=== Frecuencia SIN L2 ===
Accuracy (VA): 0.8857 | Macro-F1 (VA): 0.8856
[INFO] [LinearSVC] Binario CON L2

=== Binario CON L2 ===
Accuracy (VA): 0.9452 | Macro-F1 (VA): 0.9453
[INFO] [LinearSVC] Frecuencia CON L2

=== Frecuencia CON L2 ===
Accuracy (VA): 0.8655 | Macro-F1 (VA): 0.8651


================= RESUMEN VALIDACIÓN (80/20) =================
Binario SIN L2     -> Acc: 0.9381 | Macro-F1: 0.9381
Frecuencia SIN L2  -> Acc: 0.8857 | Macro-F1: 0.8856
Binario CON L2     -> Acc: 0.9452 | Macro-F1: 0.9453
Frecuencia CON L2  -> Acc: 0.8655 | Macro-F1: 0.8651


Ok, tenemos que la cura sí en binario es buena. Cuando tenemos frecuencia baja un poco. El binario con normalización mejora. Y creemos que también el de frecuencia pero con normalización L2 no mejora mucho. Tienen los resultados un poco peorcitos. La validación binaria funciona bien para muestras pequeñas que es de 50.000 en el vocabulario.